# Analyzing CIA Factbook Data
In this project, I will be working with [CIA World Factbook](https://www.cia.gov/the-world-factbook/) data, which includes statistics about all countries on Earth.

Let's use the code to connect Jupyter Notebook to database file.


In [2]:
# In case if you are using SQL in Jupyter for a first time, use this code. 
# It will install all necessary packages. 
# !conda install -yc conda-forge ipython-sql

In [10]:
%%capture 
%load_ext sql 
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

This code returns information on the tables in the database. 

In [11]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Overview of the Data
Let's begin by observing first 5 rows of the data. 

In [12]:
%%sql
select *
    from facts 
    limit 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The Factbook contains important demographic data such as:
 - **name** - the name of the country. 
 - **area** - the country's total area (both land and water).
 - **area_land** - the country's land area in square kilometers.
 - **area_water** - the country's water area in square kilometers.
 - **population** - the country's population
 - **population_growth** - the annual population growth rate of the country, as a percentage. 
 - **birth_rate** - the country's birth rate, or the number of births per year per 1,000 people.
 - **death_rate** - the country's death rate, or the number of death per year per 1,000 people.
     

## Summary Statistics
Continue by calculating some summary statistics and observe what they tell us.

In [13]:
%%sql 
select MIN(population) as min_population, 
    MAX(population) as max_population, 
    MIN(population_growth) as min_population_growth, 
    MAX(population_growth) as max_population_growth 
    from facts

Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


Interesting facts about calculated summary statistics:
 - There's a country with a population of 0
 - There's also a country with a population of 7256490011 (7.2+ billion of people)
 
 I will use subqueries to zoom in on just these countries _without_ using the specific values.
 

## Exploring Outliers
First, I will write a query that return countries with the minimum population.

In [14]:
%%sql 
select *
    from facts 
    where population = (select MIN(population) from facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


This table only contains a row for Antarctica, which explains population of 0, as there are no inhabitants in Antarctica. CIA webpage about Antarctica supports my findings. ![](https://camo.githubusercontent.com/6e15f6abc5b22dd52b087ba616738d5f6dd39fc666d445608f0d7c933892e5f9/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f64712d636f6e74656e742f3235372f66625f616e74617263746963612e706e67) 


Now, let's find countries with the maximum population.

In [15]:
%%sql
select * 
    from facts 
    where population  = (select MAX(population) from facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


It can be seen that there is also a row for the whole World, which could explain 7.2 billion of people that we found earlier.

Knowing this, I should recalculate summary statistics of the data without including a row for the whole World.

In [16]:
%%sql
select MIN(population) as min_population, 
    MAX(population) as max_population, 
    MIN(population_growth) as min_population_growth, 
        MAX(population_growth) as max_population_growth
    from facts
    where name <> 'World'

Done.


min_population,max_population,min_population_growth,max_population_growth
0,1367485388,0.0,4.02


The minimum population is still 0, but the maximum population is around 1.4 billion people (1367485388). Let's find out which country has the the highest population.

In [17]:
%%sql 
select * 
    from facts 
    where name <> 'World'
    ORDER BY population desc
    limit 1

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44


China is the country with approximately 1.4 billion of people, which is a maximum population according to this data. 

Now I want to find out which countries have the highest and lowest growth rates according to the data.

This code returns country with the highest annual population growth according to the data.

In [19]:
%%sql 
select * 
    from facts
    where name <> 'World'
    ORDER BY population_growth desc
    limit 1

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47


South Sudan is the country with the highest annual population growth rate. (4%)

This code returns country with the lowest annual population growth according to the data. 

In [22]:
%%sql 
    select * 
    from facts 
    where population_growth = (select MIN(population_growth) from facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
190,vt,Holy See (Vatican City),0,0,0,842,0.0,None,None,None
200,ck,Cocos (Keeling) Islands,14,14,0,596,0.0,None,None,None
207,gl,Greenland,2166086,2166086,None,57733,0.0,14.48,8.49,5.98
238,pc,Pitcairn Islands,47,47,0,48,0.0,None,None,None


Given output reveals areas with the lowest annual population growth rates. 
 - Holy See (Vatican City): 0%
 - Cocos Islands: 0%
 - Greenland: 0%
 - Pitcairn Islands: 0%

## Exploring Average Population and Area
Now I will calculate average values for `population` and `area` columns.

In [23]:
%%sql
select AVG(population) as average_population,
    AVG(area) as average_area
    from facts 
    where name <> 'World'

Done.


average_population,average_area
32242666.56846473,555093.546184739


According to results, the average population is around 32 million people and average area is 555 thousand square kilometers.

## Finding Densely Populated Countries
I will build the query to find countries which are densely populated. I will identify countries that have the following: 
- **Above-average values for population.**
- **Below-average values for area.**


In [24]:
%%sql 
select * 
    from facts 
    where population > (select AVG(population) 
                        from facts 
                        where name <> 'World')
    AND area < (select AVG(area) 
                from facts 
                where name <> 'World')

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Some of these countries in the table are known to be densely populated, so I can consider the result as accurate.

## Further Population and Area Data Analysis
There are also few interesting questions that should be answered by in this project. These questions are: 
- **1. Which countries have a higher death rate than birth rate?**
- **2. Which countries have the highest population/area ratio, and how does it compare to list I found in the previous screen?**

Let's answer the first question. First I need to identify the maximum and minimum annual death rates in the data.

In [26]:
%%sql
select MIN(death_rate) as min_death_rate, 
MAX(death_rate) as max_death_rate
    from facts

Done.


min_death_rate,max_death_rate
1.53,14.89


Minimum annual death rate is 1.53 = ~2 deaths per year.

Maximum annual death rate is 14.89 = ~15 deaths per year.

Given code identifies country with the lowest annual death rate.

In [28]:
%%sql 
select * 
    from facts 
    where death_rate = (select MIN(death_rate) 
                        from facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39


Qatar is the country with the minimum annual death rate - 1.53 = ~2 deaths per year.

Let's find a country with the highest annual death rate.

Code below identifies the country with the highest annual death rate.

In [30]:
%%sql 
select * 
    from facts 
    where death_rate = (select MAX(death_rate) 
                        from facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
98,lt,Lesotho,30355,30355,0,1947701,0.32,25.47,14.89,7.36


Lesotho is the country in South Africa that has the highest annual death rate - 14.89 = ~15 deaths per year.

Now I will find top 5 countries which have higher annual death rates than birth rates. 

In [33]:
%%sql 
select * 
    from facts
    where death_rate > birth_rate 
    ORDER BY death_rate desc;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33
116,md,Moldova,33851,32891,960,3546847,1.03,12.0,12.59,9.67
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6


Here are top five countries with annual death rates being higher than birth rate. Ukraine is the country with higher death rate- around 15 deaths to 11 births annually per 1000 people.

To conclude the project, let's identify which countries have the highest `population/area` ratio

In [35]:
%%sql 
select name, 
AVG(population)/AVG(area) as population_area_ratio
from facts
group by name
order by AVG(population)/AVG(area) DESC

Done.


name,population_area_ratio
Macau,21168.964285714286
Monaco,15267.5
Singapore,8141.279770444763
Hong Kong,6445.041516245487
Gaza Strip,5191.819444444444
Gibraltar,4876.333333333333
Bahrain,1771.8592105263158
Maldives,1319.6409395973155
Malta,1310.01582278481
Bermuda,1299.9259259259259


Macau is the country that has highest mean population to area ratio = ~21169.